In [15]:
import osmnx as ox
import networkx as nx
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import pandas as pd
import pickle

import dgl
from dgl.nn import GraphConv
import torch
from torch import nn
import torch.nn.functional as F

### Data Extraction

In [20]:
data = all_data 

In [21]:
data = data.dropna()
data.head()

,Experiment,Town,County,State,Treatment,Search_Place,Graph
0,"(AlAMU, 1875)",Huntsville,Madison,Alabama,1,"Huntsville, Alabama","(56922621, 56923776, 56923781, 56923787, 56924..."
1,"(AlAMU, 1875)",Montgomery,Montgomery,Alabama,0,"Montgomery, Alabama","(58878326, 58878327, 58878330, 58878370, 58878..."
3,"(AubU, 1856)",Auburn,Lee,Alabama,1,"Auburn, Alabama","(56844738, 56844741, 56845079, 56845082, 56845..."
4,"(AubU, 1856)",Florence,Lauderdale,Alabama,0,"Florence, Alabama","(56540270, 56540280, 56540305, 56540703, 56540..."
5,"(AubU, 1856)",Talladega,Talladega,Alabama,0,"Talladega, Alabama","(52315445, 52318540, 52326293, 52387532, 52398..."


In [22]:
def get_nodes(graphs):
    
    all_features_list = []
    
    for G in graphs:
        nodes = list(G.nodes)
        degree_tensor = torch.tensor(list(G.degree()))
        in_degree_tensor = torch.tensor(list(G.in_degree()))
        all_features_tensor = torch.cat([degree_tensor.unsqueeze(1), in_degree_tensor.unsqueeze(1)], dim=1)
        all_features_list.append(all_features_tensor)

    combined_feature_tensor = torch.cat(all_features_list, dim=0)
    return combined_feature_tensor

In [23]:
def get_edges(graphs):
    
    all_sparse_tensors = []
    
    for G in graphs:
        adj_coo = nx.to_scipy_sparse_array(G, dtype=float, format='coo')
        
        # Extract the row indices, column indices, and data
        row, col = adj_coo.row, adj_coo.col
        data = adj_coo.data
        
        # Convert to a PyTorch sparse COO tensor
        indices = torch.tensor([row, col], dtype=torch.int64)
        values = torch.tensor(data, dtype=torch.float32)
        sparse_tensor = torch.sparse_coo_tensor(indices, values, adj_coo.shape)
        
        all_sparse_tensors.append(sparse_tensor)
        
    return all_sparse_tensors

In [25]:
node_features = get_nodes(data["Graph"])

In [27]:
edge_features = get_edges(data["Graph"])

In [28]:
edge_features

[tensor(indices=tensor([[   0,    0,    2,  ..., 9308, 9308, 9308],
                        [  37, 8767,    1,  ..., 7766, 7756, 9307]]),
        values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
        size=(9309, 9309), nnz=23204, layout=torch.sparse_coo),
 tensor(indices=tensor([[   0,    1,    1,  ..., 9838, 9838, 9839],
                        [   1,    0,    2,  ..., 8248, 9839, 9838]]),
        values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
        size=(9840, 9840), nnz=24840, layout=torch.sparse_coo),
 tensor(indices=tensor([[   0,    1,    1,  ..., 2360, 2361, 2362],
                        [   1,    0, 1168,  ..., 2359, 1820, 1076]]),
        values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
        size=(2363, 2363), nnz=5713, layout=torch.sparse_coo),
 tensor(indices=tensor([[   0,    0,    0,  ..., 2339, 2339, 2340],
                        [ 904,    1, 1065,  ..., 2336, 2338, 2338]]),
        values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
        size=(2341, 2341), nnz=6488, 

### Data Preprocessing

In [5]:
class AttentionPool(nn.Module):
      """ 
      
      Attention-based pooling layer for graph classification.
      Handles graphs of different sizes. 
      
      """

      def __init__(self, units):
        super(AttentionPool, self).__init__()
        self.units = units
        
        self.attention_dense = nn.Linear(units, units)
        
        self.score_dense = nn.Linear(units, 1)

      def forward(self, node_features, adj_matrix):
        # Calculate attention scores
        attention_inputs = self.attention_dense(node_features)
        attention_logits = torch.matmul(attention_inputs, attention_inputs.t())
        # Mask out self-attention
        attention_logits -= torch.eye(attention_logits.size(0)) * 1e9
        # Might change to softmax, to normalze weightos over the logits in the attention pool
        #attention_weights = F.sigmoid(self.score_dense(attention_logits))
        attention_weights = F.softmax(attention_logits, dim=-1)
        
        # Apply attention to node features
        pooled_features = torch.matmul(attention_weights, node_features)

        return pooled_features

In [6]:
def pool_features(node_features, edge_matrices):
    pooled_features_list = []
    for node_features_batch, adj_matrix_batch in zip(node_features, edge_matrices):
        # 
        units = node_features_batch.shape[1] # Adjust units based on the input feature size
        attention_pool = AttentionPool(units=units)
        pooled_features = attention_pool(node_features_batch, adj_matrix_batch)
        pooled_features_list.append(pooled_features)
    return pooled_features_list

In [ ]:
# ME CODE
node_features = [n.float() for n in node_features]

In [29]:
test_node = node_features[0:5]
test_edge = edge_features[0:5]
test_pool = pool_features(test_node, test_edge)

RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float

### Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [2]:
class GCNConv(nn.Module):
    """Graph Convolutional Layer"""

    def __init__(self, in_features, out_features):
        super(GCNConv, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, node_features, adj_matrix):
        # Normalize adjacency matrix
        adj_matrix = F.normalize(adj_matrix, dim=1, p=1, eps=1e-9)
        # Transform features with the linear layer
        node_features_transformed = self.linear(node_features)
        # Perform graph convolution
        x = torch.matmul(adj_matrix, node_features_transformed)
        return x

In [3]:
class GraphCNN(nn.Module):
      """Graphical CNN model for network classification."""
    
      def __init__(self, in_features, hidden_features):
        super(GraphCNN, self).__init__()
        self.gcn1 = GCNConv(in_features, hidden_features)
        self.gcn2 = GCNConv(hidden_features, hidden_features)
        self.attention_pool = AttentionPool(hidden_features)
        self.fc1 = nn.Linear(hidden_features, 64)
        self.fc2 = nn.Linear(64, 2)
    
      def forward(self, node_features, adj_matrix):
        x = self.gcn1(node_features, adj_matrix)
        x = F.relu(x)
        x = self.gcn2(x, adj_matrix)
        x = F.relu(x)
        x = self.attention_pool(x, adj_matrix)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        output = F.sigmoid(self.fc2(x))
        return output
    

In [100]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 50
hidden_features = 128

# Initialize the model
model = GraphCNN(in_features=5, hidden_features=hidden_features)  # Adjust in_features
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(node_features_train, adj_matrix_train)
    loss = criterion(output, labels_train)
    loss.backward()
    optimizer.step()
    
    # Print loss periodically
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on test data
model.eval()
with torch.no_grad():
    output = model(node_features_test, adj_matrix_test)
    _, predicted = torch.max(output, 1)
    accuracy = accuracy_score(labels_test, predicted)
    print(f'Test Accuracy: {accuracy:.4f}')


Epoch [10/50], Loss: 0.6618
Epoch [20/50], Loss: 0.6513
Epoch [30/50], Loss: 0.6173
Epoch [40/50], Loss: 0.5862
Epoch [50/50], Loss: 0.5743
Test Accuracy: 0.6000


In [12]:
learning_rate = 0.001
num_epochs = 50
hidden_features = 128

model = GraphCNN(in_features=5, hidden_features=hidden_features) 

dict_model = torch.load("data/training/graphcnn_model.pth")
model.load_state_dict(dict_model)

<All keys matched successfully>

In [16]:
all_data = pd.read_pickle("data/training/Graph_State_Uni.pkl")

In [13]:
model.eval()

GraphCNN(
  (gcn1): GCNConv(
    (linear): Linear(in_features=5, out_features=128, bias=True)
  )
  (gcn2): GCNConv(
    (linear): Linear(in_features=128, out_features=128, bias=True)
  )
  (attention_pool): AttentionPool(
    (attention_dense): Linear(in_features=128, out_features=128, bias=True)
    (score_dense): Linear(in_features=128, out_features=1, bias=True)
  )
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)